In [1]:
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
from keras.layers  import  Input, Flatten, Dense,  Dropout

In [3]:
from keras.models import Model

In [4]:
from keras.optimizers  import SGD

In [5]:
from keras.datasets import mnist

In [6]:
import cv2

In [7]:
import h5py as h5py

In [8]:
import numpy as np

In [9]:
ishape=224

In [10]:
model_vgg = VGG16(include_top = False, weights = 'imagenet', 
                  input_shape = (ishape, ishape, 3))

In [11]:
for layer in model_vgg.layers:
    layer.trainable = False

In [12]:
model = Flatten(name = 'flatten')(model_vgg.output)

In [13]:
model = Dense(10, activation = 'softmax')(model)

In [14]:
model_vgg_mnist_pretrain = Model(model_vgg.input, model, 
                                 name = 'vgg16_pretrain')

In [15]:
model_vgg_mnist_pretrain.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [16]:
sgd = SGD(lr = 0.05, decay = 1e-5)

In [17]:
model_vgg_mnist_pretrain.compile(loss = 'categorical_crossentropy', 
                                 optimizer = sgd, metrics = ['accuracy'])

In [18]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [19]:
X_train = [cv2.cvtColor(cv2.resize(i, (ishape, ishape)),
                        cv2.COLOR_GRAY2BGR) for i in X_train]

In [ ]:
X_train = np.concatenate([arr[np.newaxis] for arr in X_train]).astype(
'float32')

In [ ]:
X_test = [cv2.cvtColor(cv2.resize(i, (ishape, ishape)), 
                      cv2.COLOR_GRAY2BGR) for i in X_test]

In [ ]:
X_test = np.concatenate([arr[np.newaxis] for arr in X_test]).astype(
'float32')

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = X_train/255

In [ ]:
X_test = X_test/255

In [ ]:
np.where(X_train[0]!=0)

In [ ]:
def tran_y(y):
    y_ohe = np.zeros(10)
    y_ohe[y] = 1
    return y_ohe

In [ ]:
y_train_ohe = np.array([tran_y(y_train[i]) for i in range(len(y_train))])

In [ ]:
y_test_ohe = np.array([tran_y(y_test[i]) for i in range(len(y_test))])

In [ ]:
model_vgg_mnist_pretrain.fit(X_train, y_train_ohe, validation_data =
            (X_test, y_test_ohe), epochs = 200, batch_size = 128)